# GenAI Workshop
## Lesson 2: Prompt Engineering

This lesson is intended to improve your prompt engineering skills.

During this lesson you will learn how to ...
* use a system prompt to define model behaviour
* extend system prompt to create workflows
* specify output format



### Set up the environment 

Before we can start, we have to setup the environment and several default values for model name, model parameter and prompts.  

In [ ]:
import os
from google import genai
from google.genai import types
import json
from pydantic import BaseModel
from enum import Enum
import time
from pathlib import Path
from google.colab import userdata


# Check runtime environment to make sure we are running in a colab environment. 
if os.getenv("COLAB_RELEASE_TAG"):
   COLAB = True
   print("Running on COLAB environment.") 
else:
   COLAB = False
   print("WARNING: Running on LOCAL environment.")

In [ ]:
# Clone the data repository into colab
!git clone https://github.com/openknowledge/workshop-genai-data.git
PROCESSED_DATA_PATH = "/content/workshop-genai-data/processed/gutenberg/"
BOOK_DB = PROCESSED_DATA_PATH + "books.db"

In [ ]:
# Initialize Google GenAI Client API with GOOGLE_API_KEY to be able to call the model.
# Note: GEMINI_API_KEY must be set as COLAB userdata before!
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
# Double check key settings by printing it out (or at least it length). 
if GOOGLE_API_KEY: 
    print(f' GOOGLE_API_KEY set with a length of {len(GOOGLE_API_KEY)}')
else: 
    print(f' ERROR: GOOGLE_API_KEY not set correctly!')

### Definition of convenient functions  

The three following methods will simplify to work with the GEMINI genai model.
For details see function documentation.   

In [ ]:
# Set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-2.0-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.9 
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200 
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT:str = " "

class MimeType(Enum):
    """
    Enum for MIME types.
    """
    JSON = "application/json"


class ResponseFormat(BaseModel):
    """
    Response format model for Gemini API.
    """
    response_mime_type: MimeType
    response_schema: type
    
history = [
    types.Part.from_bytes(
        data=Path(BOOK_DB).read_bytes(),
        mime_type='text/plain',
    )
]

def clear_history():
    """
    Clear the history of the conversation.
    """
    global history
    history = [
        types.Part.from_bytes(
            data=Path(BOOK_DB).read_bytes(),
            mime_type='text/plain',
        )
    ]

def generate_bookstore_bot_completion(
        user_prompt : str,
        response_schema: BaseModel | None = None,
        system_prompt: str = DEFAULT_SYSTEM_PROMPT,
        model_name: str = DEFAULT_MODEL,
        verbose: bool = False
        ): 
    """
    Call the GenAI model with function declarations and return the response.
    Args:
        user_prompt (str): The prompt to send to the model.
        response_format (ResponseFormat): The format of the response.
        system_prompt (str): The system prompt to use.
        model_name (str): The name of the model to use.
        verbose (bool): If True, print the response.
    Returns:
        str: The response from the model.
    """
    global history

    # Append file content if provided
    user_content = types.Content(
        role='user',
        parts=[types.Part.from_text(text=user_prompt)]
    )

    # Update history with user content
    history.append(user_content)

    # Configure response format
    response_mime_type = None
    if response_schema:
        response_mime_type = MimeType.JSON.value

    config = types.GenerateContentConfig(
        system_instruction=system_prompt,
        response_schema=response_schema,
        response_mime_type=response_mime_type,
    )

    # Send request with function declarations
    response = client.models.generate_content(
        model=model_name,
        contents=history,
        config=config,
    )

    # Update history with assistant content
    bot_content = types.ModelContent(response.text)

    history.append(user_content)
    history.append(bot_content)

    if verbose:
        print(f"User Prompt: {user_prompt}")
        print(f"Assistant Response: {response.text}")
    
    return response

In [ ]:
def print_completion_result(completion_result, full:bool = False):
    
    """ Prints out the completion.    
    
    Parameters
    ----------
    completion_result : str
        A instance of GenerateContentResponse representing a completion 
    full : bool, optional [default: False]
    Whether to print all details of the completion or only the text. Defaults to False
    
    """            
    print(f'\nANSWER of genAI model: \n')
    if full:
        print(completion_result)
    else: 
        print(completion_result.text) 

In [ ]:
def order_book(isbn:str):
    """ Orders a book by its ISBN number. 
    
    Parameters
    ----------
    isbn : str
        The ISBN number of the book to order.
    
    Returns 
    -------
    order_status : str
        The status of the order. 
    """
    # Simulate ordering the book
    print(f"Ordering book with ISBN: {isbn}")
    # Print "." every 0.5 seconds
    for _ in range(5):
        print(".", end="", flush=True)
        time.sleep(0.5)
    print()
    if isbn == "978-3-51593-12345-6":
        print("Success: You ordered 'A Study in Scarlet'!")
        print("You completed this exercise successfully!")
    else:
        print("Error: Unknown ISBN number.")

### Important notice: History
In this (and only this) exercise, we implemented a history feature, so that the conversation can keep on going. If you mess up, use the following function to clear the history.

In [ ]:
# Clean the chat history
clear_history()

### Exercise 01: Basic interaction
Your task is to create a simple chatbot, which is able to order a book based on the description of a book.  
Imagine the following situation:
A customer of a bookstore wants to buy a book, but does not remember its title.  
The customer might ask an employee: *"I'm looking for this book, where Sherlock Holmes and Watson meet the first time."*
In response an employee will use his/her extensive knowledge of books and say: "That's "A Study in Scarlet"! Should I place an order for this book?"
  
The employee's reaction described here, should now be done by a chatbot. Your task is to provide a system prompt for this bot. 
The bot should:  
* Respond to the customer by naming the book title, year of publication and a short summary of the book.
* In addition the bot should ask, if the book should be ordered.  

**Hints**: The creators of the bot (aka teacher of this workshop) already provided knowledge about books to the bot. You do not need to worry about this.


 

In [ ]:
# Do not change the user prompt.
user_prompt = "I'm looking for this book, where Sherlock Holmes and Watson meet the first time."

# TODO: Define the respective system prompt.
system_prompt = "TODO"

# We provide some knowledge to the model about the book contents.
response = generate_bookstore_bot_completion(user_prompt=user_prompt,system_prompt=system_prompt)
print_completion_result(response)

### Exercise 02: Extend the process
Extend the system prompt even further.
After the bot provided the information about the book and asks if it should be ordered, the *customer might say*: "Yes, please!" As an result, the bot will order the book. This process is finished by the bot replying with an object like {"isbn": "978-4-23050-12345-6"}. This object represents the payload for ordering the book using an API.
Your task is to extend the system prompt in order to achieve this behaviour.

In [ ]:
# TODO: Define the respective system prompt.
system_prompt = "TODO"

In [ ]:
# Clear the chat history
clear_history()

# The customer is looking for a book.
customer_initial_prompt = "I'm looking for this book, where Sherlock Holmes and Watson meet the first time."
print(f'Customer:\n {customer_initial_prompt}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_initial_prompt, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

# The customer wants to order the book.
customer_answer = "Yes, I like to order the book."
print(f'Customer:\n {customer_answer}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_answer, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')


In [ ]:
# TODO: Order the book by using the ISBN number provided by the bot.
isbn = "TODO"
order_book(isbn=isbn)

### Exercise 03: Give choices
After the initial customer request, the bot should give the user another choice. Instead of ordering the book, the bot can provide an url to the ebook version of the book. Update the system prompt in order to achieve this.

In [ ]:
# TODO Update the system prompt to include the option to provide an url to the ebook instead of ordering the book.
system_prompt = "TODO"

In [ ]:
# Clear the chat history
clear_history()

# The customer is looking for a book.
customer_initial_prompt = "I'm looking for this book, where Sherlock Holmes and Watson meet the first time."
print(f'Customer:\n {customer_initial_prompt}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_initial_prompt, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

# The customer wants to to have the link to the ebook.
customer_answer = "I love ebooks. Why should I order a book, if I get an ebook for free. Please provide the link!"
print(f'Customer:\n {customer_answer}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_answer, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

In [ ]:
# Test if the old process of ordering a book still works

# Clear the chat history
clear_history()

# The customer is looking for a book.
customer_initial_prompt = "I'm looking for this book, where Sherlock Holmes and Watson meet the first time."
print(f'Customer:\n {customer_initial_prompt}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_initial_prompt, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')

# The customer wants to order the book.
customer_answer = "Please order the book. I love spending money for stuff I can get for free!"
print(f'Customer:\n {customer_answer}\n')

# The bookstore bot answers the customer.
response = generate_bookstore_bot_completion(user_prompt=customer_answer, system_prompt=system_prompt)
print(f'Bookstore bot:\n {response.text}\n')